In [1]:
import pandas as pd
import numpy as np

print('Версия pandas:', pd.__version__)

Версия pandas: 2.1.4


# Переиндексация структур в pandas

## Фундаментальный метод выравнивания данных в pandas
## Методы .reindex & .reindex_like объектов Series и DataFrame. 

_Метод .reindex() является фундаментальным методом выравнивания данных в Pandas. Он используется для реализации почти всех других функций, основанных на функциональности выравнивания индексных меток. Повторная индексация означает согласование данных с заданным набором меток вдоль определенной оси. Это позволяет достичь нескольких целей:_

#### 1. Изменяет порядок существующих данных в соответствии с новым набором меток
#### 2. Вставляет маркеры отсутствующих значений (NA) в места записей, где не существовало данных для этой метки
#### 3. Если необходимо, заполнит данные для отсутствующих меток с помощью логики (очень важно для работы с данными временных рядов)

#### Пример 1.

In [2]:
s = pd.Series(np.random.randint(1, 100, 5), index=['a', 'b', 'c', 'd', 'e'])
s

a    20
b    36
c    27
d    26
e    78
dtype: int32

In [3]:
# создаем новый список new_index_list
new_index_list=['e', 'b', 'f', 'd']
# передаем в метод reindex new_index_list с частично совпадающими метками
s.reindex(new_index_list)
# метод reindex помещает NA/NaN в места, не имеющие значения в предыдущем индексе

e    78.0
b    36.0
f     NaN
d    26.0
dtype: float64

## Заполняем пропуски NaN

    fill_value=nan - значение, используемое для отсутствующих значений. 
    По умолчанию NaN, но может быть любым "совместимым" значением.

#### Пример 2.

In [4]:
# используем fill_value=0
s.reindex(new_index_list, fill_value=0)

e    78
b    36
f     0
d    26
dtype: int32

## Выбираем ось индекса (строк или столбцов)

    labels=None - новые метки/индекс для соответствия оси, заданной axis.
    
    axis=None - может быть либо именем оси 'index'/'columns', либо числом 0/1. 
    Объект Series этот аргумент не использует.

    index=None - новые метки для индекса. 
    Предпочтительно объект Index, чтобы избежать дублирования данных.

    columns=None - новые метки для столбцов. 
    Предпочтительно объект Index, чтобы избежать дублирования данных. 
    Объект Series этот аргумент не использует.

In [5]:
df = pd.DataFrame({'one': pd.Series(np.random.randint(1, 100, 3), index=['a', 'b', 'c']),
     'two': pd.Series(np.random.randint(1, 100, 4), index=['a', 'b', 'c', 'd']),
     'three': pd.Series(np.random.randint(1, 100, 3), index=['b', 'c', 'd']),})
df

,one,two,three
a,95.0,44,NaN
b,9.0,36,39.0
c,18.0,79,1.0
d,NaN,97,72.0


#### Пример 3.1

In [6]:
display(df.reindex(labels=['a', 'b', 'f'], axis=0),
        df.reindex(labels=['a', 'b', 'f'], axis='index'))

,one,two,three
a,95.0,44.0,NaN
b,9.0,36.0,39.0
f,NaN,NaN,NaN


,one,two,three
a,95.0,44.0,NaN
b,9.0,36.0,39.0
f,NaN,NaN,NaN


#### Пример 3.2

In [7]:
display(df.reindex(labels=['two', 'four'], axis=1),
        df.reindex(labels=['two', 'four'], axis='columns'))

,two,four
a,44,NaN
b,36,NaN
c,79,NaN
d,97,NaN


,two,four
a,44,NaN
b,36,NaN
c,79,NaN
d,97,NaN


#### Пример 3.3

In [8]:
# с помощью DataFrame.reindex можно одновременно переиндексировать строки и столбцы:
df.reindex(index=['c', 'f', 'b'], columns=['three', 'two', 'one'])

,three,two,one
c,1.0,79.0,18.0
f,NaN,NaN,NaN
b,39.0,36.0,9.0


## Заполнение значений при расширении индекса
## Метод заполнения NaN. Аргумент method

    Аргумент method задает метод, используемый для заполнения NA в переиндексированном DataFrame/Series. 
    Обратите внимание: method применим только к монотонно растущим/уменьшающимся индексам.
    
    Варианты:
    None (по умолчанию): не заполнять пробелы
    pad/ffill: использовать последнее достоверное наблюдение для заполнения NaN
    backfill/bfill: использовать следующее достоверное наблюдение для заполнения NaN
    nearest: использовать ближайшие достоверные наблюдения для заполнения NaN

#### Пример 4.1

In [9]:
# создаем набор дат для индекса серии ts
d_rng = pd.date_range("1/15/2025", periods=8)
ts = pd.Series(np.random.randint(0, 100, 8), index=d_rng)
# берем несколько строк из структуры ts для создания серии ts2
ts2 = ts.iloc[[0, 3, 6]]

display(ts, ts2)

2025-01-15    24
2025-01-16    61
2025-01-17    85
2025-01-18     7
2025-01-19     5
2025-01-20    85
2025-01-21    75
2025-01-22    51
Freq: D, dtype: int32

2025-01-15    24
2025-01-18     7
2025-01-21    75
Freq: 3D, dtype: int32

In [10]:
# переиндексовываем структуру ts2 с помощью индексов структуры ts
ts2.reindex(ts.index)

2025-01-15    24.0
2025-01-16     NaN
2025-01-17     NaN
2025-01-18     7.0
2025-01-19     NaN
2025-01-20     NaN
2025-01-21    75.0
2025-01-22     NaN
Freq: D, dtype: float64

#### Пример 4.2 - Заполнение пропусков ВПЕРЕД

In [11]:
print(ts2.reindex(ts.index))
# pad/ffill - заполнение значений вперед
ts2.reindex(ts.index, method='ffill')
# такой же результат будет при последовательном использовании методов .reindex() и .ffill(): ts2.reindex(ts.index).ffill()

2025-01-15    24.0
2025-01-16     NaN
2025-01-17     NaN
2025-01-18     7.0
2025-01-19     NaN
2025-01-20     NaN
2025-01-21    75.0
2025-01-22     NaN
Freq: D, dtype: float64


2025-01-15    24
2025-01-16    24
2025-01-17    24
2025-01-18     7
2025-01-19     7
2025-01-20     7
2025-01-21    75
2025-01-22    75
Freq: D, dtype: int32

#### Пример 4.3 - Заполнение пропусков НАЗАД

In [12]:
print(ts2.reindex(ts.index))
# bfill/backfill - заполнение значений в обратном направлении
ts2.reindex(ts.index, method='bfill')
# такой же результат будет при последовательном использовании методов .reindex() и .bfill(): ts2.reindex(ts.index).bfill()

2025-01-15    24.0
2025-01-16     NaN
2025-01-17     NaN
2025-01-18     7.0
2025-01-19     NaN
2025-01-20     NaN
2025-01-21    75.0
2025-01-22     NaN
Freq: D, dtype: float64


2025-01-15    24.0
2025-01-16     7.0
2025-01-17     7.0
2025-01-18     7.0
2025-01-19    75.0
2025-01-20    75.0
2025-01-21    75.0
2025-01-22     NaN
Freq: D, dtype: float64

#### Пример 4.4 - Заполнение пропусков БЛИЖАЙШИМ

In [13]:
print(ts2.reindex(ts.index))
# nearest - заполнение от ближайшего значения
ts2.reindex(ts.index, method='nearest')

2025-01-15    24.0
2025-01-16     NaN
2025-01-17     NaN
2025-01-18     7.0
2025-01-19     NaN
2025-01-20     NaN
2025-01-21    75.0
2025-01-22     NaN
Freq: D, dtype: float64


2025-01-15    24
2025-01-16    24
2025-01-17     7
2025-01-18     7
2025-01-19     7
2025-01-20    75
2025-01-21    75
2025-01-22    75
Freq: D, dtype: int32

## Лимиты на заполнение при переиндексации

#### Аргумент limit

#### Пример 5.1

In [14]:
ts2.reindex(ts.index, method="ffill", limit=1)

2025-01-15    24.0
2025-01-16    24.0
2025-01-17     NaN
2025-01-18     7.0
2025-01-19     7.0
2025-01-20     NaN
2025-01-21    75.0
2025-01-22    75.0
Freq: D, dtype: float64

#### Аргумент tolerance

#### abs(index[indexer] - target) <= tolerance

_Значение аргумента tolerance может быть скалярным, которое применяет один и тот же допуск ко всем значениям, или объектом list-like, который применяет переменный допуск к каждому элементу. List-like объект включает список, кортеж, массив, серию и должен быть того же размера, что и индекс, а его dtype должен точно соответствовать типу индекса._

#### Пример 5.2.1 

In [15]:
# производим одну замену (указываем "один день", соответвующий типу данных)
display(ts2,
        ts2.reindex(ts.index, method="ffill", tolerance='1 day'))

2025-01-15    24
2025-01-18     7
2025-01-21    75
Freq: 3D, dtype: int32

2025-01-15    24.0
2025-01-16    24.0
2025-01-17     NaN
2025-01-18     7.0
2025-01-19     7.0
2025-01-20     NaN
2025-01-21    75.0
2025-01-22    75.0
Freq: D, dtype: float64

#### Пример 5.2.2

In [16]:
# в первом случае замена 2 пропусков, в остальных - только один пропуск. 
# при этом указан соответствующий тип данных - '1 day', '2 days'
# размер переденного списка совпадает по размеру с ts.index

ts2.reindex(ts.index, method="ffill", 
            tolerance=['1 day', '1 day', '2 days', '1 day', '1 day', '1 day', '1 day', '1 day'])

2025-01-15    24.0
2025-01-16    24.0
2025-01-17    24.0
2025-01-18     7.0
2025-01-19     7.0
2025-01-20     NaN
2025-01-21    75.0
2025-01-22    75.0
Freq: D, dtype: float64

#### Пример 5.2.3

In [17]:
# обращаем внимание на тип данных, когда используем аргумент tolerance
# пропуски не заполнились, поскольку в tolerance передана просто 1, а не 1 day
ts2.reindex(ts.index, method="ffill", tolerance=1)

2025-01-15    24.0
2025-01-16     NaN
2025-01-17     NaN
2025-01-18     7.0
2025-01-19     NaN
2025-01-20     NaN
2025-01-21    75.0
2025-01-22     NaN
Freq: D, dtype: float64

#### Пример 5.2.4

In [18]:
s_1 = pd.Series(['a', 'e', 'f', 'j'], index=[1001, 1005, 1006, 1010])
display(s_1, s_1.reindex([1001,1002,1003,1004,1005,1006,1007,1008,1009,1010], 
                    method='ffill', tolerance=2))

1001    a
1005    e
1006    f
1010    j
dtype: object

1001      a
1002      a
1003      a
1004    NaN
1005      e
1006      f
1007      f
1008      f
1009    NaN
1010      j
dtype: object

_Значение NaN, присутствующее в исходном DataFrame не будет заполнено ни одной из схем распространения значений. Это связано с тем, что заполнение при переиндексации не рассматривает значения исходного DataFrame, а только сравнивает исходный и требуемый индексы. Если необходимо заполнить прорпуски, присутствующие в исходном DataFrame, необходимо использовать метод DataFrame.fillna()._

#### Пример 5.3.1

In [19]:
# если в исходной структуре есть Nan, аргумент method не заполнит эти пропуски
s_2 = pd.Series(['a', np.nan, 'f', 'j'], index=[1001, 1005, 1006, 1010])
display(s_2, s_2.reindex([1001,1002,1003,1004,1005,1006,1007,1008,1009,1010], 
                    method='ffill'))

1001      a
1005    NaN
1006      f
1010      j
dtype: object

1001      a
1002      a
1003      a
1004      a
1005    NaN
1006      f
1007      f
1008      f
1009      f
1010      j
dtype: object

#### Пример 5.3.2

In [20]:
# используем fillna + reindex
s_2.fillna('e').reindex([1001,1002,1003,1004,1005,1006,1007,1008,1009,1010], 
                    method='ffill')

1001    a
1002    a
1003    a
1004    a
1005    e
1006    f
1007    f
1008    f
1009    f
1010    j
dtype: object

## Метод reindex_like - переиндексация для выравнивания по другому объекту

In [21]:
df2 = pd.DataFrame({'one': pd.Series(np.random.randint(1, 100, 3), index=['a', 'b', 'f']),
     'two': pd.Series(np.random.randint(1, 100, 4), index=['a', 'b', 'c', 'd']),
     'four': pd.Series(np.random.randint(1, 100, 3), index=['b', 'c', 'f']),})

display(df, df2)

,one,two,three
a,95.0,44,NaN
b,9.0,36,39.0
c,18.0,79,1.0
d,NaN,97,72.0


,one,two,four
a,87.0,93.0,NaN
b,57.0,34.0,79.0
c,NaN,25.0,79.0
d,NaN,80.0,NaN
f,14.0,NaN,25.0


#### Пример 6.

In [22]:
display(df.reindex_like(df2),
        df2.reindex_like(df))

,one,two,four
a,95.0,44.0,NaN
b,9.0,36.0,NaN
c,18.0,79.0,NaN
d,NaN,97.0,NaN
f,NaN,NaN,NaN


,one,two,three
a,87.0,93.0,NaN
b,57.0,34.0,NaN
c,NaN,25.0,NaN
d,NaN,80.0,NaN
